In [1]:
import torch
import torch.nn.functional as F
import numpy as np

In [83]:
from numpy import dtype


tokens = [*[str(i) for i in range(10)], "MIN", "MAX", "MED", "SM", "[", "]"]
padding_index = len(tokens)
lines = open("train_preprocessed.txt").readlines()

Y = []
X = []
for line in lines:
    y, x = line.split(",")
    y = int(y)
    x = [tokens.index(t) for t in x.strip().split(" ")]
    Y.append(y)
    X.append(x)

I = [i for i, x in enumerate(X) if len(x) <= 32]
XX = []
YY = torch.tensor([Y[i] for i in I])
for i in I:
    XX.append([*[16]*(32 - len(X[i])), *X[i]])
XX = torch.tensor(XX)
XX = F.one_hot(XX).float()


In [85]:
train_ds = torch.utils.data.TensorDataset(XX, YY)
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=64, shuffle=True, drop_last=True)

In [86]:
x, y = next(iter(train_dl))

In [87]:
x.shape, y.shape

(torch.Size([64, 32, 17]), torch.Size([64]))

In [94]:
gru = torch.nn.GRU(17, 128, batch_first=True)
lin = torch.nn.Linear(128, 10)

In [101]:
output, h = gru(x)
y_hat = lin(h.squeeze())

In [104]:
F.cross_entropy(y_hat, y)

tensor(2.3120, grad_fn=<NllLossBackward0>)

In [132]:
class MyModel(torch.nn.Module):
    def __init__(self, hidden_size=128, num_layers=1):
        super().__init__()
        self.gru = torch.nn.GRU(input_size=17, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.lin = torch.nn.Linear(hidden_size, 10)

    def forward(self, x):
        _, h = self.gru(x)
        y_hat = self.lin(h)
        return y_hat

In [133]:
epochs = 3

model = MyModel(num_layers=2)
opt = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(epochs):
  losses = []
  for batch, (x, y) in enumerate(train_dl):
    y_hat = model(x)[-1]
    loss = F.cross_entropy(y_hat, y)
    loss.backward()
    opt.step()
    opt.zero_grad()
    with torch.no_grad():
      loss = loss.detach().clone()
      losses.append(loss.item())
      if (batch + 1) % 100 == 0:
        print(f"epoch {epoch}, batch {batch + 1}, loss {loss:.4f}")

epoch 0, batch 100, loss 2.0359
epoch 0, batch 200, loss 2.1141
epoch 0, batch 300, loss 1.8010
epoch 0, batch 400, loss 1.7544
epoch 0, batch 500, loss 1.3858
epoch 0, batch 600, loss 1.2675
epoch 0, batch 700, loss 1.4372
epoch 0, batch 800, loss 1.1133
epoch 0, batch 900, loss 1.2422
epoch 1, batch 100, loss 1.1496
epoch 1, batch 200, loss 1.3715
epoch 1, batch 300, loss 1.1647
epoch 1, batch 400, loss 0.9206
epoch 1, batch 500, loss 1.2092
epoch 1, batch 600, loss 1.1762
epoch 1, batch 700, loss 1.1974
epoch 1, batch 800, loss 1.1496
epoch 1, batch 900, loss 0.9355
epoch 2, batch 100, loss 0.9140
epoch 2, batch 200, loss 1.0696
epoch 2, batch 300, loss 0.8702
epoch 2, batch 400, loss 1.1084
epoch 2, batch 500, loss 0.9034
epoch 2, batch 600, loss 1.1110
epoch 2, batch 700, loss 0.9939
epoch 2, batch 800, loss 0.9626
epoch 2, batch 900, loss 0.9408
